# Quantum Arithmetic - Adder, Multiplier, and Exponentiation (Q# code)

This notebook implements quantum adder (Draper Adder), multipler, exponentiation, and also these modulus version (Beauregard Adder) in Q#.

See [here](https://tsmatz.wordpress.com/2019/05/22/quantum-computing-modulus-add-subtract-multiply-exponent/) for mathematical background.

> Note : For the purpose of your understanding, here I implement algorithm very straightforward without any optimization. (See "[Circuit for Shor's algorithm using 2n + 3 qubits](https://arxiv.org/pdf/quant-ph/0205095v3.pdf)" or "[Fast Quantum Modular Exponentiation Architecture](https://arxiv.org/pdf/1207.0511.pdf)" for more optimized algorithms.)

*back to [index](https://github.com/tsmatz/quantum-algorithms-qsharp)*

In [1]:
import qsharp

Preparing Q# environment...


For preparation, implement **Quantum Fourier Transform (shortly, QFT)** as follows. (The code is same as [03-phase-estimation.ipynb](./03-phase-estimation.ipynb).)

See [here](https://tsmatz.wordpress.com/2019/04/26/quantum-computing-qsharp-quantum-fourier-transform-and-phase-estimation/) for details about this code.

> Note : You can also use built-in ```Microsoft.Quantum.Canon.QFT()``` instead.

In [2]:
%%qsharp
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

operation QFTImpl (qs : Qubit[]) : Unit is Adj + Ctl
{
    body (...)
    {
        let nQubits = Length(qs);

        for i in 0 .. nQubits - 1
        {
            H(qs[i]);
            for j in i + 1 .. nQubits - 1
            {
                Controlled R1Frac([qs[j]], (1, j - i, qs[i]));
            }
        }

        Microsoft.Quantum.Canon.SwapReverseRegister(qs);
    }
}

## Quantum Adder (Draper Adder)

Implement : |x⟩ |y⟩ -> |x⟩ |x + y mod 2^n⟩ where n = Length(x) = Length(y)<br>
with [Drapper algorithm](https://arxiv.org/pdf/1411.5949.pdf).

In [3]:
# declare function
TestQuantumAdd: any = None

In [4]:
%%qsharp
operation QuantumAdd (x : Qubit[], y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(x);
    QFTImpl(y);
    for i in 0 .. n - 1 {
        for j in 0 .. (n - 1) - i {
            Controlled R1Frac([x[i + j]], (2, j + 1, (y)[(n - 1) - i]));
        }
    }
    Adjoint QFTImpl(y);
}

operation TestQuantumAdd (a : Int, b : Int, n : Int) : Int {
    mutable resultArray = new Result[n];

    use (x, y) = (Qubit[n], Qubit[n]) {
        // create qubit's array from integer a (ex: 3 -> |011⟩)
        mutable array1 = new Bool[n];
        mutable tempInt1 = a;
        for idxBit in 0 .. n - 1 {
            set array1 w/= ((n - 1) - idxBit) <- tempInt1 % 2 == 0 ? false | true;
            set tempInt1 = tempInt1 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array1[idx]) {
                X(x[idx]);
            }
        }

        // create qubit's array from integer b (ex: 3 -> |011⟩)
        mutable array2 = new Bool[n];
        mutable tempInt2 = b;
        for idxBit in 0 .. n - 1 {
            set array2 w/= ((n - 1) - idxBit) <- tempInt2 % 2 == 0 ? false | true;
            set tempInt2 = tempInt2 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array2[idx]) {
                X(y[idx]);
            }
        }

        // apply Drapper adder
        QuantumAdd(x, y);

        // measure and reset
        for idx in 0 .. n - 1 {
            set resultArray w/= idx <- MResetZ(y[idx]);
        }
        for idx in 0 .. n - 1 {
            Reset(x[idx]);
        }
    }

    // get integer's result from measured array (ex : |011⟩ -> 3)
    let resultBool = Microsoft.Quantum.Convert.ResultArrayAsBoolArray(resultArray);
    mutable resultInt = 0;
    for idx in 0 .. n - 1 {
        if(resultBool[idx]) {
            set resultInt = resultInt + (2 ^ ((n - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(13,27): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(17,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(30,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


Run and check with Python code.

In [5]:
n = 3
b = 3
for a in range(3, 8):
    res = TestQuantumAdd.simulate(a=a, b=b, n=n)
    print("{} + {} is {} (mod {})".format(a, b, res, 2**n))
n = 3
a = 7
for b in range(4, 8):
    res = TestQuantumAdd.simulate(a=a, b=b, n=n)
    print("{} + {} is {} (mod {})".format(a, b, res, 2**n))

3 + 3 is 6 (mod 8)
4 + 3 is 7 (mod 8)
5 + 3 is 0 (mod 8)
6 + 3 is 1 (mod 8)
7 + 3 is 2 (mod 8)
7 + 4 is 3 (mod 8)
7 + 5 is 4 (mod 8)
7 + 6 is 5 (mod 8)
7 + 7 is 6 (mod 8)


## Quantum Subtractor (Adjoint Adder)

In [6]:
# declare function
TestQuantumSub: any = None

In [7]:
%%qsharp
operation TestQuantumSub (a : Int, b : Int, n : Int) : Int {
    mutable resultArray = new Result[n];

    use (x, y) = (Qubit[n], Qubit[n]) {
        // create qubit's array from integer a (ex: 3 -> |011⟩)
        mutable array1 = new Bool[n];
        mutable tempInt1 = a;
        for idxBit in 0 .. n - 1 {
            set array1 w/= ((n - 1) - idxBit) <- tempInt1 % 2 == 0 ? false | true;
            set tempInt1 = tempInt1 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array1[idx]) {
                X(x[idx]);
            }
        }

        // create qubit's array from integer b (ex: 3 -> |011⟩)
        mutable array2 = new Bool[n];
        mutable tempInt2 = b;
        for idxBit in 0 .. n - 1 {
            set array2 w/= ((n - 1) - idxBit) <- tempInt2 % 2 == 0 ? false | true;
            set tempInt2 = tempInt2 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array2[idx]) {
                X(y[idx]);
            }
        }

        // apply adjoint of Drapper adder
        // Implement : |x⟩ |y⟩ -> |x⟩ |y - x⟩
        Adjoint QuantumAdd(x, y);

        // measure and reset
        for idx in 0 .. n - 1 {
            set resultArray w/= idx <- MResetZ(y[idx]);
        }
        for idx in 0 .. n - 1 {
            Reset(x[idx]);
        }
    }

    // get integer's result from measured array (ex : |011⟩ -> 3)
    let resultBool = Microsoft.Quantum.Convert.ResultArrayAsBoolArray(resultArray);
    mutable resultInt = 0;
    for idx in 0 .. n - 1 {
        if(resultBool[idx]) {
            set resultInt = resultInt + (2 ^ ((n - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(2,27): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(6,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(19,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [8]:
n = 3
b = 5
for a in range(3, 8):
    res = TestQuantumSub.simulate(a=a, b=b, n=n)
    print("{} - {} is {} (mod {})".format(b, a, res, 2**n))

5 - 3 is 2 (mod 8)
5 - 4 is 1 (mod 8)
5 - 5 is 0 (mod 8)
5 - 6 is 7 (mod 8)
5 - 7 is 6 (mod 8)


## Quantum Adder by Classical Numeric

Implement : |x⟩ -> |x + b mod 2^n⟩ for some integer b

In [9]:
# declare function
TestQuantumAddByNumber: any = None

In [10]:
%%qsharp
operation QuantumAddByNumber (x : Qubit[], b : Int) : Unit is Adj + Ctl {
    let n = Length(x);

    // apply Draper adder for numeric
    QFTImpl(x);
    for i in 0 .. n - 1 {
        for j in 0 .. (n - 1) - i {
            if(not((b / 2^((n - 1) - (i + j))) % 2 == 0)) {
                R1Frac(2, j + 1, (x)[(n - 1) - i]);
            }
        }
    }
    Adjoint QFTImpl(x);
}

operation TestQuantumAddByNumber (a : Int, b : Int, n : Int) : Int {
    mutable resultArray = new Result[n];

    use x = Qubit[n] {
        // create qubit's array from integer a (ex: 3 -> |011⟩)
        mutable array1 = new Bool[n];
        mutable tempInt1 = a;
        for idxBit in 0 .. n - 1 {
            set array1 w/= ((n - 1) - idxBit) <- tempInt1 % 2 == 0 ? false | true;
            set tempInt1 = tempInt1 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array1[idx]) {
                X(x[idx]);
            }
        }

        // apply Draper adder for numeric
        QuantumAddByNumber(x, b);

        // measure and reset
        for idx in 0 .. n - 1 {
            set resultArray w/= idx <- MResetZ(x[idx]);
        }
    }

    // get integer's result from measured array (ex : |011⟩ -> 3)
    let resultBool = Microsoft.Quantum.Convert.ResultArrayAsBoolArray(resultArray);
    mutable resultInt = 0;
    for idx in 0 .. n - 1 {
        if(resultBool[idx]) {
            set resultInt = resultInt + (2 ^ ((n - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(17,27): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(21,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [11]:
n = 3
b = 3
for a in range(3, 8):
    res = TestQuantumAddByNumber.simulate(a=a, b=b, n=n)
    print("{} + {} is {} (mod {})".format(a, b, res, 2**n))
n = 3
a = 7
for b in range(4, 8):
    res = TestQuantumAddByNumber.simulate(a=a, b=b, n=n)
    print("{} + {} is {} (mod {})".format(a, b, res, 2**n))

3 + 3 is 6 (mod 8)
4 + 3 is 7 (mod 8)
5 + 3 is 0 (mod 8)
6 + 3 is 1 (mod 8)
7 + 3 is 2 (mod 8)
7 + 4 is 3 (mod 8)
7 + 5 is 4 (mod 8)
7 + 6 is 5 (mod 8)
7 + 7 is 6 (mod 8)


## Quantum Multiplier

Implement : |y⟩ -> |a y mod 2^n⟩ for some integer a

> Note : Integer "a" and modulus must be co-prime number. (Because this operator must be controlled. Otherwise ```InverseModI()``` will raise an error.)

In [12]:
# declare function
TestQuantumMultiply: any = None

In [14]:
%%qsharp
operation QuantumMultiply (a : Int, y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(y);
    let a_mod = a % (2^n);

    use s = Qubit[n] {
        // start |y⟩ |0⟩

        // apply adder by repeating "a" (integer) times
        for r in 0 .. a_mod - 1 {
            QuantumAdd(y, s);
        }
        // now |y⟩ |a y mod N⟩

        // swap first register and second one by tuple
        Microsoft.Quantum.Canon.ApplyToEachCA(SWAP, Microsoft.Quantum.Arrays.Zipped(y, s));
        // now |a y mod N⟩ |y⟩

        // reset all s qubits !
        // but it's tricky because we cannot use "Reset()" since here is controlled operator.
        let a_inv = InverseModI(a_mod, 2^n);
        for r in 0 .. a_inv - 1 {
            Adjoint QuantumAdd(y, s);
        }
    }
}

operation TestQuantumMultiply (a : Int, b : Int, n : Int) : Int {
    mutable resultArray = new Result[n];

    use y = Qubit[n] {
        // create qubit's array from integer b (ex: 3 -> |011⟩)
        mutable array2 = new Bool[n];
        mutable tempInt2 = b;
        for idxBit in 0 .. n - 1 {
            set array2 w/= ((n - 1) - idxBit) <- tempInt2 % 2 == 0 ? false | true;
            set tempInt2 = tempInt2 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array2[idx]) {
                X(y[idx]);
            }
        }

        // apply multiplier
        QuantumMultiply(a, y);

        // measure and reset
        for idx in 0 .. n - 1 {
            set resultArray w/= idx <- MResetZ(y[idx]);
        }
    }

    // get integer's result from measured array (ex : |011⟩ -> 3)
    let resultBool = Microsoft.Quantum.Convert.ResultArrayAsBoolArray(resultArray);
    mutable resultInt = 0;
    for idx in 0 .. n - 1 {
        if(resultBool[idx]) {
            set resultInt = resultInt + (2 ^ ((n - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(28,27): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(32,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [15]:
n = 3
a = 5
for b in range(3, 8):
    res = TestQuantumMultiply.simulate(a=a, b=b, n=n)
    print("{} x {} is {} (mod {})".format(a, b, res, 2**n))

5 x 3 is 7 (mod 8)
5 x 4 is 4 (mod 8)
5 x 5 is 1 (mod 8)
5 x 6 is 6 (mod 8)
5 x 7 is 3 (mod 8)


## Quantum Exponentiation

Implement : |x⟩ -> |a^x mod 2^n⟩ for some integer a

> Note : Integer "a" and modulus must be co-prime number. (Because this invokes ```QuantumMultiply()```.)

In [16]:
# declare function
TestQuantumExponent: any = None

In [17]:
%%qsharp
operation QuantumExponent (a : Int, x : Qubit[]) : Unit {
    let n = Length(x);
    use s = Qubit[n] {
        // set |s⟩ = |1⟩
        X(s[n - 1]);

        // apply decomposition elements
        for idx in 0 .. n - 1 {
            Controlled QuantumMultiply([x[idx]], (a^(2^((n-1) - idx)), s));
        }

        // swap |x⟩ and |s⟩
        Microsoft.Quantum.Canon.ApplyToEachCA(SWAP, Microsoft.Quantum.Arrays.Zipped(x, s));

        // Reset s
        for idx in 0 .. n - 1 {
            Reset(s[idx]);
        }
    }
}

operation TestQuantumExponent (a : Int, b : Int, n : Int) : Int {
    mutable resultArray = new Result[n];

    use x = Qubit[n] {
        // create qubit's array from integer b (ex: 3 -> |011⟩)
        mutable array2 = new Bool[n];
        mutable tempInt2 = b;
        for idxBit in 0 .. n - 1 {
            set array2 w/= ((n - 1) - idxBit) <- tempInt2 % 2 == 0 ? false | true;
            set tempInt2 = tempInt2 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array2[idx]) {
                X(x[idx]);
            }
        }

        // apply multiplier
        QuantumExponent(a, x);

        // measure and reset
        for idx in 0 .. n - 1 {
            set resultArray w/= idx <- MResetZ(x[idx]);
        }
    }

    // get integer's result from measured array (ex : |011⟩ -> 3)
    let resultBool = Microsoft.Quantum.Convert.ResultArrayAsBoolArray(resultArray);
    mutable resultInt = 0;
    for idx in 0 .. n - 1 {
        if(resultBool[idx]) {
            set resultInt = resultInt + (2 ^ ((n - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(23,27): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(27,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [18]:
n = 3
a = 3
for b in range(3, 8):
    res = TestQuantumExponent.simulate(a=a, b=b, n=n)
    print("{} ^ {} is {} (mod {})".format(a, b, res, 2**n))

3 ^ 3 is 3 (mod 8)
3 ^ 4 is 1 (mod 8)
3 ^ 5 is 3 (mod 8)
3 ^ 6 is 1 (mod 8)
3 ^ 7 is 3 (mod 8)


## Arithmetic Operations (Adder, Multiplyer, Exponentiator) by Modulus N

In [19]:
# declare function
TestQuantumAddByModulus: any = None

In [20]:
%%qsharp
//
// Implement : |x⟩ |y⟩ -> |x⟩ |x+y mod N⟩ for arbitray integer N (< 2^n)
// (where N < 2^n, x < N, y < N)
//
operation QuantumAddByModulus (N : Int, x : Qubit[], y : Qubit[]) : Unit is Adj + Ctl {
    use (ancilla, cx, cy) = (Qubit(), Qubit(), Qubit()) {
        // add bit for preventing overflow
        let x_large = [cx] + x;
        let y_large = [cy] + y;
        // |x⟩ |y⟩ -> |x⟩ |x + y⟩
        QuantumAdd(x_large, y_large);
        // |y⟩ -> |y - N⟩
        Adjoint QuantumAddByNumber(y_large, N);
        // Turn on ancilla when first bit is |1⟩ (i.e, when x + y - N < 0)
        Controlled X([y_large[0]], ancilla);
        // Add N back when ancilla is |1⟩
        Controlled QuantumAddByNumber([ancilla], (y_large, N));
        // set ancilla to |0⟩
        Adjoint QuantumAdd(x_large, y_large);
        X(ancilla);
        Controlled X([y_large[0]], ancilla);
        QuantumAdd(x_large, y_large);
    }
}

//
// Implement : |y⟩ -> |a y mod N⟩ for some integer a and N
// (where N < 2^n, y < N)
//
// Important Note :
// Integer "a" and N must be co-prime number.
// (For making this operator must be controlled. Otherwise InverseModI() raises an error.)
//
operation QuantumMultiplyByModulus (N : Int, a : Int, y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(y);
    let a_mod = a % N;

    use s = Qubit[n] {
        // start |y⟩ |0⟩

        // apply adder by repeating "a" (integer) times
        for r in 0 .. a_mod - 1 {
            QuantumAddByModulus(N, y, s);
        }
        // now |y⟩ |a y mod N⟩

        // swap first register and second one by tuple
        Microsoft.Quantum.Canon.ApplyToEachCA(SWAP, Microsoft.Quantum.Arrays.Zipped(y, s));
        // now |a y mod N⟩ |y⟩

        // reset all s qubits !
        // but it's tricky because we cannot use "Reset()" since here is controlled operator.
        let a_inv = InverseModI(a_mod, N);
        for r in 0 .. a_inv - 1 {
            Adjoint QuantumAddByModulus(N, y, s);
        }
    }
}

//
// Implement : |x⟩ -> |a^x mod N⟩ for some integer a and N
// (where N < 2^n)
//
// Important Note :
// Integer "a" and N must be co-prime number.
// (Because this invokes QuantumMultiplyByModulus().)
//
operation QuantumExponentByModulus (N : Int, a : Int, x : Qubit[]) : Unit {
    let n = Length(x);
    use s = Qubit[n] {
        // set |s⟩ = |1⟩
        X(s[n - 1]);

        // apply decomposition elements
        for idx in 0 .. n - 1 {
            Controlled QuantumMultiplyByModulus([x[idx]], (N, a^(2^((n-1) - idx)), s));
        }

        // swap |x⟩ and |s⟩
        Microsoft.Quantum.Canon.ApplyToEachCA(SWAP, Microsoft.Quantum.Arrays.Zipped(x, s));

        // Reset s
        for idx in 0 .. n - 1 {
            Reset(s[idx]);
        }
    }
}

operation TestQuantumAddByModulus (a : Int, b : Int, n : Int, N : Int) : Int {
    mutable resultArray = new Result[n];

    use (x, y) = (Qubit[n], Qubit[n]) {
        // create qubit's array from integer a (ex: 3 -> |011⟩)
        mutable array1 = new Bool[n];
        mutable tempInt1 = a;
        for idxBit in 0 .. n - 1 {
            set array1 w/= ((n - 1) - idxBit) <- tempInt1 % 2 == 0 ? false | true;
            set tempInt1 = tempInt1 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array1[idx]) {
                X(x[idx]);
            }
        }

        // create qubit's array from integer b (ex: 3 -> |011⟩)
        mutable array2 = new Bool[n];
        mutable tempInt2 = b;
        for idxBit in 0 .. n - 1 {
            set array2 w/= ((n - 1) - idxBit) <- tempInt2 % 2 == 0 ? false | true;
            set tempInt2 = tempInt2 / 2;
        }
        for idx in 0 .. n - 1 {
            if(array2[idx]) {
                X(y[idx]);
            }
        }

        // apply Drapper adder
        QuantumAddByModulus(N, x, y);
        //QuantumMultiplyByModulus(N, a, y);
        //Adjoint QuantumAddByModulus(N, x, y);
        //QuantumExponentByModulus(N, a, y);

        // measure and reset
        for idx in 0 .. n - 1 {
            set resultArray w/= idx <- MResetZ(y[idx]);
        }
        for idx in 0 .. n - 1 {
            Reset(x[idx]);
        }
    }

    // get integer's result from measured array (ex : |011⟩ -> 3)
    let resultBool = Microsoft.Quantum.Convert.ResultArrayAsBoolArray(resultArray);
    mutable resultInt = 0;
    for idx in 0 .. n - 1 {
        if(resultBool[idx]) {
            set resultInt = resultInt + (2 ^ ((n - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(90,27): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(94,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(107,26): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [23]:
n = 3
N = 7
a = 3
for b in range(3, 7):
    res = TestQuantumAddByModulus.simulate(a=a, b=b, n=n, N=N)
    print("{} + {} is {} (mod {})".format(a, b, res, N))

3 + 3 is 6 (mod 7)
3 + 4 is 0 (mod 7)
3 + 5 is 1 (mod 7)
3 + 6 is 2 (mod 7)
